In [1]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

# Récupération des clés API stockées dans Google Colab
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
FINANCIAL_MODELING_PREP_API_KEY = userdata.get('FINANCIAL_MODELING_PREP_API_KEY')

import os
import requests
from openai import OpenAI



# Initialisation du client OpenAI avec la clé API
client = OpenAI(api_key=OPENAI_API_KEY)

In [3]:
import requests

def fetch_stock_data(symbol):
    """
    Récupère le prix actuel de l'action, le volume actuel, les moyennes de prix sur 50 et 200 jours, le BPA (EPS),
    le ratio cours/bénéfices (PE), et la date de la prochaine annonce des résultats pour un symbole donné.
    """
    url = f"https://financialmodelingprep.com/api/v3/quote-order/{symbol}?apikey={FINANCIAL_MODELING_PREP_API_KEY}"
    response = requests.get(url)
    data = response.json()

    try:
        stock_info = data[0]
        return {
            "symbol": symbol.upper(),
            "price": stock_info.get('price'),
            "volume": stock_info.get('volume'),
            "priceAvg50": stock_info.get('priceAvg50'),
            "priceAvg200": stock_info.get('priceAvg200'),
            "EPS": stock_info.get('eps'),
            "PE": stock_info.get('pe'),
            "earningsAnnouncement": stock_info.get('earningsAnnouncement')
        }
    except (IndexError, KeyError):
        return {"error": f"Impossible de récupérer les données pour le symbole : {symbol}"}

def fetch_company_overview(symbol):
    """
    Récupère les informations financières de base d'une entreprise donnée telles que le secteur,
    l'industrie, le nom de l'entreprise et la capitalisation boursière.
    """
    url = f"https://financialmodelingprep.com/api/v3/profile/{symbol}?apikey={FINANCIAL_MODELING_PREP_API_KEY}"
    response = requests.get(url)
    data = response.json()

    try:
        company_info = data[0]
        return {
            "symbol": company_info.get("symbol"),
            "companyName": company_info.get("companyName"),
            "marketCap": company_info.get("mktCap"),
            "industry": company_info.get("industry"),
            "sector": company_info.get("sector"),
            "website": company_info.get("website"),
            "beta": company_info.get("beta"),
            "price": company_info.get("price")
        }
    except (IndexError, KeyError):
        return {"error": f"Impossible de récupérer les informations financières pour le symbole : {symbol}"}

def fetch_income_statement(symbol):
    """
    Récupère le dernier état des résultats d'une entreprise donnée, y compris les revenus,
    le bénéfice brut, le résultat net, l'EBITDA, et le BPA (EPS).
    """
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{symbol}?period=annual&apikey={FINANCIAL_MODELING_PREP_API_KEY}"
    response = requests.get(url)
    data = response.json()

    try:
        income_info = data[0]
        return {
            "date": income_info.get("date"),
            "revenue": income_info.get("revenue"),
            "grossProfit": income_info.get("grossProfit"),
            "netIncome": income_info.get("netIncome"),
            "ebitda": income_info.get("ebitda"),
            "EPS": income_info.get("eps"),
            "EPSDiluted": income_info.get("epsdiluted")
        }
    except (IndexError, KeyError):
        return {"error": f"Impossible de récupérer les états financiers pour le symbole : {symbol}"}


In [4]:
# Définir les schémas de fonction pour OpenAI
functions = [
    {
        "name": "fetch_stock_data",
        "description": "Obtenir le prix actuel de l'action pour un symbole d'entreprise donné.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "Le symbole boursier de l'entreprise, par exemple, AAPL pour Apple."
                }
            },
            "required": ["symbol"],
        },
    },
    {
        "name": "fetch_company_overview",
        "description": "Obtenir les informations financières de base pour une entreprise donnée.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "Le symbole boursier de l'entreprise, par exemple, GOOGL pour Alphabet Inc."
                }
            },
            "required": ["symbol"],
        },
    },
    {
        "name": "fetch_income_statement",
        "description": "Obtenir le dernier état des résultats pour une entreprise donnée.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "Le symbole boursier de l'entreprise, par exemple, AMZN pour Amazon."
                }
            },
            "required": ["symbol"],
        },
    },
]


In [6]:
import json
# Assistant de Chat - Analyse Financière
MODEL_NAME = "gpt-4o-mini"

def chat_with_LLM(model_name=MODEL_NAME):
    conversation_history = []
    print("Bienvenue! Je suis votre assistant analyste financier. Posez vos questions sur les actions, états financiers ou bilans comptables.")

    while True:
        user_input = input("\nVous: ")

        # Condition pour quitter la boucle
        if user_input.lower() in ["exit", "quit"]:
            print("Assistant: Fin de la discussion !")
            break

        # Ajout de la requête de l'utilisateur dans l'historique des messages
        conversation_history.append({"role": "user", "content": user_input})

        # Appel au modèle LLM
        response = client.chat.completions.create(
            model=model_name,
            messages=conversation_history,
            functions=functions,
            function_call="auto"  # Laisser le modèle décider d'appeler une fonction
        )

        # Récupération de la réponse du modèle
        model_reply = response.choices[0].message

        # Vérification si une fonction doit être appelée
        if model_reply.function_call:
            func_name = model_reply.function_call.name
            func_args = json.loads(model_reply.function_call.arguments)

            # Appel à la fonction appropriée
            if func_name == "fetch_stock_data":
                func_response = fetch_stock_data(**func_args)
            elif func_name == "fetch_company_overview":
                func_response = fetch_company_overview(**func_args)
            elif func_name == "fetch_income_statement":
                func_response = fetch_income_statement(**func_args)
            else:
                func_response = {"error": "Fonction inconnue."}

            # Mise à jour de l'historique avec l'appel de fonction et sa réponse
            conversation_history.append(model_reply)  # Appel de fonction par l'assistant
            conversation_history.append({
                "role": "function",
                "name": func_name,
                "content": json.dumps(func_response)
            })

            # Deuxième réponse après l'appel de fonction
            final_response = client.chat.completions.create(
                model=model_name,
                messages=conversation_history,
            )

            final_reply = final_response.choices[0].message.content
            print(f"Assistant: {final_reply}")
            conversation_history.append({"role": "assistant", "content": final_reply})
        else:
            # Si aucune fonction n'est appelée, afficher directement la réponse
            print(f"Assistant: {model_reply.content}")
            conversation_history.append({"role": "assistant", "content": model_reply.content})

# Exécution de la fonction de chat
if __name__ == "__main__":
    chat_with_LLM()


Bienvenue! Je suis votre assistant analyste financier. Posez vos questions sur les actions, états financiers ou bilans comptables.

Vous: Quels sont les principaux indicateurs financiers de Google (GOOGL) ?
Assistant: Les principaux indicateurs financiers de Google, qui fait partie d'Alphabet Inc. (symbole boursier : GOOGL), sont les suivants :

- **Nom de l'entreprise** : Alphabet Inc.
- **Capitalisation boursière** : environ 1 999,76 milliards de dollars
- **Secteur** : Services de communication
- **Industrie** : Contenu et information sur Internet
- **Prix de l'action** : 161,86 $
- **Beta** : 1,038 (indicateur de la volatilité par rapport au marché)

Pour plus d'informations, vous pouvez visiter leur site web : [Alphabet Inc.](https://www.abc.xyz). 

Si vous avez d'autres questions ou si vous souhaitez des informations plus détaillées, n'hésitez pas à demander !

Vous: Quel est le prix actuel de l'action AAPL ?
Assistant: Le prix actuel de l'action d'Apple (symbole boursier : AAPL)